In [1]:
import os
from pathlib import Path
from sqlalchemy import create_engine

import xmlrpc.client
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
# api_db = os.environ.get('ODOO_DB_PRUEBA_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [2]:
with engine.connect() as conn, conn.begin():  
    ventas_enero = pd.read_sql_table('ventas_enero', conn, dtype_backend='numpy_nullable')
    ventas_febrero = pd.read_sql_table('ventas_febrero', conn, dtype_backend='numpy_nullable')
    ventas_marzo = pd.read_sql_table('ventas_marzo', conn, dtype_backend='numpy_nullable')
    ventas_abril = pd.read_sql_table('ventas_abril', conn, dtype_backend='numpy_nullable')
    ventas_mayo = pd.read_sql_table('ventas_mayo', conn, dtype_backend='numpy_nullable')
    ventas_junio = pd.read_sql_table('ventas_junio', conn, dtype_backend='numpy_nullable')
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [3]:
ventas_año = ventas_año[ventas_año['invoice_date'].dt.month < 6]

In [4]:
db_file1 = 'proveedores_oficiales.xlsx'
db_file1_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file1}'))

proveedores = pd.read_excel(db_file1_path_str)
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [5]:
db_file2 = 'codigos_sin_compra.xlsx'
db_file2_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file2}'))

codigos_sin_compra = pd.read_excel(db_file2_path_str)

In [6]:
fields_compras_doc = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras_doc = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras_doc], {'fields': fields_compras_doc})

In [7]:
data_compras_doc = []

for compra in json_compras_doc:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras_doc.append(new)

compras_doc = pd.DataFrame(data_compras_doc)
compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')

In [8]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras_doc)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [9]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id_pp'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [10]:
compras = pd.merge(compras_linea,
                   compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id')

In [11]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(proveedores['partner_id'])]).drop_duplicates('partner_id')
not check1.empty and print('Hay proveedores no calificados')
check1


Hay proveedores no calificados


,order_id,order_name,order_state,order_date,partner_id,partner_name,partner_fact_ref,partner_fact_date,capturista,vendedora
0,2981,P02967,purchase,2024-06-17 19:51:15,5170,Casa Ikeda,pendiente,NaT,Alexa Yadira Mazariegos Zunun,Brenda Luz Acosta Lopez
20,3003,P02989,purchase,2024-06-18 16:10:38,17063,Operadora de Franquicias Malabares SA de CV,CP 220,2024-06-11,Alexa Yadira Mazariegos Zunun,Alexa Yadira Mazariegos Zunun
106,2901,P02887,purchase,2024-06-12 18:03:40,17062,Aurora Yanet Cruz Amaya,E4442D02-60EB-4EA9-8B7F-EE63230835D3,2024-06-12,Alexa Yadira Mazariegos Zunun,Irma Carvajal Flores


# Pruebas

In [12]:
compras_oficiales = compras.loc[compras['oficial'] == True]

In [13]:
ventas_año['costo_producto'] = pd.NA
ventas_año['costo_order_date'] = pd.NaT
ventas_año['costo_order_line_id'] = pd.NA

for i in range(len(ventas_año)):

    minidf = (compras[
            (compras['order_date'] <= pd.to_datetime(ventas_año['invoice_date'].iloc[i]) + pd.Timedelta(days=15)) 
            & (compras['product_id_pp'] == ventas_año['product_id'].iloc[i])]
            .sort_values('order_date', ascending = False)
            )

    if not minidf.empty:
        ventas_año.iloc[i,30] = minidf['product_price'].iloc[0]
        ventas_año.iloc[i,31] = minidf['order_date'].iloc[0]
        ventas_año.iloc[i,32] = minidf['line_id'].iloc[0]

    else:
        minidf_costo_inicial = codigos_sin_compra[codigos_sin_compra['id_prod_prod'] == ventas_año['product_id'].iloc[i]]

        if not minidf_costo_inicial.empty:
            ventas_año.iloc[i,30] = minidf_costo_inicial['costo'].iloc[0]


ventas_año['costo_date_dif'] = (ventas_año['invoice_date'] - ventas_año['costo_order_date']).dt.days

In [14]:
ventas_año[ventas_año['costo_producto'].isna()].drop_duplicates('product_id')

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,prod_código,prod_línea,quantity,price_unit,discount,price_subtotal,costo_producto,costo_order_date,costo_order_line_id,costo_date_dif
0,114,F1-CC/2024/00001,2024-01-02,posted,Shop/0005,PdV,10,out_invoice,<NA>,<NA>,...,2253,8,20.00,6.47,0.00,129.40,<NA>,NaT,<NA>,NaN
1,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,3041,00FAN,3.00,36.03,0.00,108.09,<NA>,NaT,<NA>,NaN
2,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,3033,00FAN,3.00,37.98,0.00,113.94,<NA>,NaT,<NA>,NaN
3,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,4642,0SAYL,1.00,209.99,0.20,167.99,<NA>,NaT,<NA>,NaN
6,125,F1-CC/2024/00004,2024-01-02,posted,Shop/0007,PdV,17,out_invoice,<NA>,<NA>,...,1881,126,2.00,6.30,0.00,12.60,<NA>,NaT,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47141,80847,F1-CC/2024/07212,2024-05-13,posted,PdV CSL/7183,PdV,16882,out_invoice,<NA>,<NA>,...,25893,0SAYG,2.00,906.78,0.10,"1,632.20",<NA>,NaT,<NA>,NaN
47570,81608,F1-CC/2024/07279,2024-05-14,posted,PdV CSL/7252,PdV,17044,out_invoice,<NA>,<NA>,...,2299,00TRU,1.00,40.64,0.00,40.64,<NA>,NaT,<NA>,NaN
49425,85207,F1-CC/2024/07566,2024-05-18,posted,PdV CSL/7543,PdV,17696,out_invoice,<NA>,<NA>,...,24309,0MAKA,10.00,24.38,0.00,243.80,<NA>,NaT,<NA>,NaN
49700,85657,F1-CC/2024/07610,2024-05-20,posted,PdV CSL/7588,PdV,17789,out_invoice,<NA>,<NA>,...,1769,00TRU,1.00,721.16,0.00,721.16,<NA>,NaT,<NA>,NaN
